#統計推論 Homework4

##應數碩一 M112040034 李祐瑄

-------------------------------------------------------------------------
###1.使用恰當的permuation test方法完成投影片 w12_permutation_test第14頁的練習。 

設定虛無假設為：兩手投球距離並無明顯差異

In [ ]:
import numpy as np
import pandas as pd
#建立w12投影片p14的資料
data = {
    '學童': [1, 2, 3, 4, 5, 6, 7],
    '慣用手': [77.3, 95.1, 97.2, 94.5, 83.6, 90.2, 84.3],
    '非慣用手': [74.6, 83.4, 80.6, 90.2, 78.7, 83.6, 76.2]
}
df = pd.DataFrame(data)
df

,學童,慣用手,非慣用手
0,1,77.3,74.6
1,2,95.1,83.4
2,3,97.2,80.6
3,4,94.5,90.2
4,5,83.6,78.7
5,6,90.2,83.6
6,7,84.3,76.2


In [ ]:
x = np.array(df['慣用手'])
y = np.array(df['非慣用手'])
#計算實際差異值
diff = np.mean(x) - np.mean(y)
#進行permutation test
np.random.seed(2023)
n_permutations = 10000  # 設置次數
diff_permutations = np.zeros(n_permutations)

for i in range(n_permutations):
    combined = np.concatenate([x, y])
    np.random.shuffle(combined)
    diff_permutations[i] = np.mean(combined[:len(x)]) - np.mean(combined[len(x):])

#計算p-value
p_value = (np.abs(diff_permutations) >= np.abs(diff)).mean()
#計算標準差
sd = np.std(diff_permutations)/np.sqrt(n_permutations)
#計算信賴區間
lb = diff - (1.96*sd)#下界
ub = diff + (1.96*sd)#上界

#判斷
is_significant = (0 < lb) or (0 > ub)
print("confidence interval:","[",round(lb,4),',',round(ub,4),']')
print("p-value:",round(p_value,4))
if is_significant == 0:
  print("不拒絕虛無假設")
else:
  print("拒絕虛無假設")

confidence interval: [ 7.7654 , 7.9203 ]
p-value: 0.0454
拒絕虛無假設


也就是說兩手投球距離有明顯差異

-------------------------------------------------------------------------
###2.使用具備predictive mean matching的插補方法，將附件檔案中NA的部分填上恰當的數值。 

In [ ]:
import pandas as pd
#匯入資料
evo = pd.read_csv('https://bit.ly/45pgzLU')
evo

,ID,y.net,gdp,mme,fme,fhe,mhe,lexp,lintr,gedy,Iy,gcony,lblakp,pol,ttrad
0,1,0.041503,7.330405,0.132,0.067,0.005,0.022,3.879500,0.113761,0.03815,0.18982,0.06014,NaN,0.083250,0.100145
1,2,0.024244,6.591674,0.439,0.089,0.004,0.013,3.788725,0.817125,0.02675,0.06633,0.12550,0.04355,0.000000,0.013976
2,3,0.007135,6.432940,0.176,0.017,0.005,0.015,3.683867,1.406029,0.03860,0.10182,0.27716,0.04355,0.100000,-0.002062
3,4,0.046417,6.938284,0.748,0.324,0.052,0.140,3.875359,0.085350,0.04890,0.13103,0.24286,0.04355,0.338500,0.016127
4,5,0.001259,6.766192,0.551,0.135,0.008,0.034,3.832980,0.496294,0.04095,0.08459,0.08226,0.50310,0.150000,0.029546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,157,0.007499,7.950855,0.609,0.228,0.018,0.094,4.050044,-0.298647,0.03120,0.26035,0.02151,0.16125,0.558200,-0.035275
157,158,0.021316,9.104869,1.920,1.860,0.235,0.468,4.276666,3.752114,0.05305,0.19046,0.07219,0.00000,0.151778,0.014021
158,159,0.016189,9.360483,3.061,2.720,0.299,1.011,4.273884,8.794455,0.04900,0.29621,0.03279,0.00000,0.000000,-0.031711
159,160,0.007641,9.258273,3.235,2.875,0.449,0.670,4.276666,8.479855,0.04705,0.25006,0.06585,0.00000,0.000000,0.003371


In [ ]:
#確認哪些columns有NA值
evo.isnull().any()

ID        False
y.net      True
gdp       False
mme       False
fme       False
fhe       False
mhe       False
lexp       True
lintr     False
gedy      False
Iy        False
gcony     False
lblakp     True
pol       False
ttrad     False
dtype: bool

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import IterativeImputer

def impute_missing_values(data):
    #建立插補器
    imputer = IterativeImputer(estimator=RandomForestRegressor(), initial_strategy='mean')
    #將DataFrame轉換為 NumPy 陣列
    array_data = data.values
    #執行插補
    imputed_data = imputer.fit_transform(array_data)
    #將NumPy陣列轉換回DataFrame
    imputed_df = pd.DataFrame(imputed_data, columns=data.columns)
    return imputed_df

imputed_data = impute_missing_values(evo)
imputed_data['ID'] = imputed_data['ID'].astype(int)

#確認插補後的資料沒有NA值
for i in range(len(list(imputed_data.isnull().any()))):
  decision = 0
  if list(imputed_data.isnull().any())[i] == True:
    decision = 1
  else:
    break
if decision == True:
  print("資料未能成功插補")
else:
  print("資料中已無NA值，成功插補")
imputed_data

資料中已無NA值，成功插補


,ID,y.net,gdp,mme,fme,fhe,mhe,lexp,lintr,gedy,Iy,gcony,lblakp,pol,ttrad
0,1,0.041503,7.330405,0.132,0.067,0.005,0.022,3.879500,0.113761,0.03815,0.18982,0.06014,0.203842,0.083250,0.100145
1,2,0.024244,6.591674,0.439,0.089,0.004,0.013,3.788725,0.817125,0.02675,0.06633,0.12550,0.043550,0.000000,0.013976
2,3,0.007135,6.432940,0.176,0.017,0.005,0.015,3.683867,1.406029,0.03860,0.10182,0.27716,0.043550,0.100000,-0.002062
3,4,0.046417,6.938284,0.748,0.324,0.052,0.140,3.875359,0.085350,0.04890,0.13103,0.24286,0.043550,0.338500,0.016127
4,5,0.001259,6.766192,0.551,0.135,0.008,0.034,3.832980,0.496294,0.04095,0.08459,0.08226,0.503100,0.150000,0.029546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,157,0.007499,7.950855,0.609,0.228,0.018,0.094,4.050044,-0.298647,0.03120,0.26035,0.02151,0.161250,0.558200,-0.035275
157,158,0.021316,9.104869,1.920,1.860,0.235,0.468,4.276666,3.752114,0.05305,0.19046,0.07219,0.000000,0.151778,0.014021
158,159,0.016189,9.360483,3.061,2.720,0.299,1.011,4.273884,8.794455,0.04900,0.29621,0.03279,0.000000,0.000000,-0.031711
159,160,0.007641,9.258273,3.235,2.875,0.449,0.670,4.276666,8.479855,0.04705,0.25006,0.06585,0.000000,0.000000,0.003371


-------------------------------------------------------------------------
###3.根據2. 填補後的資料，以其中的 lexp 為輸出變數建立多元迴歸模型，並透過 forward selection 方法選擇合適的變數。

In [ ]:
import pandas as pd
import statsmodels.api as sm

imputed_data.set_index('ID')
#將輸出變數'lexp'與其他資料分開
y = imputed_data['lexp']
X = imputed_data.drop('lexp', axis=1)

#forward selection
def forward_selection(X, y):
    included = []
    while True:
        changed = False #用來標機該特徵是否已經被選擇
        excluded = list(set(X.columns) - set(included)) #未選擇的特徵
        #初始化
        best_pval = 1.0
        best_feature = None
        
        for feature in excluded:
          #建構回歸模型計算p-value
            model = sm.OLS(y, sm.add_constant(X[included + [feature]])).fit()
            pval = model.pvalues[feature]
            if pval < best_pval:
                best_pval = pval #更新最佳p-value
                best_feature = feature #更新最佳特徵
        # 若新的p-value小於閾值，則將該變數加入選擇的變數列表中
        if best_feature is not None and best_pval < 0.05:
            included.append(best_feature) #將選到的特徵存入list
            changed = True #標記已選擇
        
        if not changed:
            break
    
    return included

selected_features = forward_selection(X, y)
print("Selected Features:", selected_features)

Selected Features: ['gdp', 'ttrad', 'y.net', 'lblakp', 'lintr', 'fme', 'Iy']


透過forward selection方法根據p-value選擇出以下變數

'gdp', 'ttrad', 'y.net', 'lblakp', 'lintr', 'fme', 'Iy'